# Params

In [ ]:
path = '../data/VPC/'
cores_path = path + 'Ki67/'
target_path = path + 'multiscale_patches_Ki67/'

# path = '../data/Biomax_TMA/' # path to the dataset
# cores_path = path + 'HnE/' # path to the images
# target_path = path + 'multiscale_patches/' # where to save the patches

# Import

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
from skimage import io
import random
from skimage.transform import resize

# Utils

In [ ]:
def padder(img, size):
    h, w, _ = img.shape
    pad_h = -h % size
    pad_w = -w % size
    return np.pad(img, ((0,pad_h), (0,pad_w), (0,0)), 'constant', constant_values=255)

def resizer(img, size):
    return (resize(img, (size, size, 3), anti_aliasing=False)*255).astype('uint8')

def patch_extractor_image(img_path, size, size_path):
    img = io.imread(img_path)
    img = padder(img, size)
    img_20 = np.pad(img, ((size//2,size//2), (size//2,size//2), (0,0)), 'constant', constant_values=255)
    img_10 = np.pad(img_20, ((size,size), (size,size), (0,0)), 'constant', constant_values=255)
    h, w, c = img.shape
    for i in range(h // size):
        for j in range(w // size):
            h_point = i*size
            w_point = j*size
            # 40x
            patch_40 = img[h_point:h_point+size, w_point:w_point+size]
            io.imsave(size_path + '40/' + str(w_point) + '_' + str(h_point) + '.png', patch_40)
            # 20x
            patch_20 = img_20[h_point:h_point+2*size, w_point:w_point+2*size]
            patch_20 = resizer(patch_20, size)
            io.imsave(size_path + '20/' + str(w_point) + '_' + str(h_point) + '.png', patch_20)
            # 10x
            patch_10 = img_10[h_point:h_point+4*size, w_point:w_point+4*size]
            patch_10 = resizer(patch_10, size)
            io.imsave(size_path + '10/' + str(w_point) + '_' + str(h_point) + '.png', patch_10)
            
def directory_maker(path):
    if not os.path.exists(path):
        os.mkdir(path)
            
def path_constructor(img_file, target_path, size, magnifications):
    # directories target_path/slidecore/size/magnification/w_h.png
    core_path = target_path + img_file[:-4] + '/'
    directory_maker(core_path)
        
    size_path = core_path + str(size) + '/'
    directory_maker(size_path)
    
    for mag in magnifications:
        mag_path = size_path + str(mag) + '/'
        directory_maker(mag_path)
        
    return size_path

# Extracting

In [ ]:
def patch_extractor_dataset(dataset_path, size, target_path):
    magnifications = [10, 20, 40] # hard coded in the extractor
    # storing image files path
    img_files = os.listdir(dataset_path)
    for img_file in img_files:
        size_path = path_constructor(img_file, target_path, size, magnifications)
        img_path = dataset_path + img_file
        patch_extractor_image(img_path, size, size_path)

In [ ]:
patch_extractor_dataset(cores_path, 512, target_path)

In [ ]:
img_files = os.listdir(cores_path)
print(img_files[0][:-4])

In [ ]:
img_file = os.listdir(cores_path)[5]
print(img_file)
img = io.imread(cores_path + img_file)
plt.imshow(img)
h, w, _ = img.shape
print(h, w)
print('slide: {} - core: {}'.format(int(img_file[5:8]), int(img_file[-7:-4])))


In [ ]:
padded = padder(img, 550)
print(padded.shape)
plt.imshow(padded)

In [ ]:
size = 512
temp_path = target_path + str(size) + '/'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
